FaceRec Evaluation 3: MeMAD - YLE Dataset
====================================


#### NOT COMPLETED

This notebook serves as evaluation for the face recognition system.

It contains the code for 1. **create a ground truth** and 2. **test the system on it**.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import re
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from frame_collector import FrameCollector
from src.utils.media_fragment import convert_to_seconds_npt
from src.connectors import memad_connector as memad
from src.connectors import limecraft_connector as limecraft

## Part 1: Create a Ground Truth

The [MeMAD Knowledge Graph](http://data.memad.eu) contains information about the presence of people in particular video segments.

The following cells retrieve the 6 people with the highest number of appearence, for then collecting all segments (+ media uri and start/end time) involving them.

For clarity, we will call _media_ an entire video resource (e.g. a mpeg4 file), while with _segment_ a temporal fragment of variable length, to not be confused with _shots_. See definitions of MediaResource and Part in the [ebucore ontology](https://www.ebu.ch/metadata/ontologies/ebucore/)

Problem: Yle has no parts

In [ ]:
ENDPOINT = "http://data.memad.eu/sparql-endpoint"
sparql = SPARQLWrapper(ENDPOINT)
sparql.setReturnFormat(JSON)

In [89]:
with open('memad_people.rq') as file:
    query = file.read()

sparql.setQuery(query)
people = sparql.query().convert()["results"]["bindings"]
for x in people:
    for k in x:
        x[k] = x[k]['value']
df_people = pd.DataFrame.from_dict(people)
df_people['nb'] = df_people['nb'].apply(pd.to_numeric)
df_people

,c,label,nb
0,http://data.memad.eu/agent/esa-saari,Esa Saari,94
1,http://data.memad.eu/agent/johan-hellman,Johan Hellman,90
2,http://data.memad.eu/agent/birgitta-norrgard,Birgitta Norrgård,89
3,http://data.memad.eu/agent/dani-stromback,Dani Strömbäck,88
4,http://data.memad.eu/agent/jussi-pernaa,Jussi Pernaa,85
...,...,...,...
1867,http://data.memad.eu/agent/riittasisko-jukkala...,Riittasisko Jukkala-Benisch,1
1868,http://data.memad.eu/agent/gunnbritt-estlander,Gunnbritt Estlander,1
1869,http://data.memad.eu/agent/reijo-leskinen,Reijo Leskinen,1
1870,http://data.memad.eu/agent/mohammed-ibrahim,Mohammed Ibrahim,1


Select top 10 celebrities

In [90]:
people_uri = []
people_labels = []
iterator = df_people.itertuples()
while len(people_uri) < 10:
    p = next(iterator)
    if p.c not in people_uri:
        people_uri.append(p.c)
        people_labels.append(p.label)
list(zip(people_uri, people_labels))

[('http://data.memad.eu/agent/esa-saari', 'Esa Saari'),
 ('http://data.memad.eu/agent/johan-hellman', 'Johan Hellman'),
 ('http://data.memad.eu/agent/birgitta-norrgard', 'Birgitta Norrgård'),
 ('http://data.memad.eu/agent/dani-stromback', 'Dani Strömbäck'),
 ('http://data.memad.eu/agent/jussi-pernaa', 'Jussi Pernaa'),
 ('http://data.memad.eu/agent/camilla-boije', 'Camilla Boije'),
 ('http://data.memad.eu/agent/ann-helen-pafs-streng', 'Ann-Helen Påfs-Streng'),
 ('http://data.memad.eu/agent/camilla-forsen-strom', 'Camilla Forsén-Ström'),
 ('http://data.memad.eu/agent/christina-staffans', 'Christina Staffans'),
 ('http://data.memad.eu/agent/antti-kuusiniemi', 'Antti Kuusiniemi')]

In [101]:
with open('memad_program_by_person.rq') as file:
    query = file.read()
    
from src.utils.media_fragment import convert_to_seconds_npt
def pad_spaces(text):
    pad = 25 - len(text)
    return text + ''.join([' ' for _ in range(0,pad)])
    
def get_segments(person, name):
    q = query.replace('?person', '<%s>' % person)
    sparql.setQuery(q)
    results = sparql.query().convert()["results"]["bindings"]
    for r in results:
        for p in r:
            value = r[p]['value']
            if p in ['start', 'end']:
                value = convert_to_seconds_npt(re.sub(r"T(\d{2}:\d{2}:\d{2}).+", "\g<1>", value))
            if p == 'prop':
                value = re.split(r"[/#]", value)[-1]
            r[p] = value
        r['person'] = name
        r['person_uri'] = person
    
    results = [r for r in results if 'start' in r and r['start'] < r['end']]
    print('- %s\t%s'%(pad_spaces(name), len(results)))
    
    
    return pd.DataFrame.from_dict(results) 

print('Num. results per person:')
df = pd.concat([get_segments(uri, name) 
                     for uri, name in tqdm(zip(people_uri, people_labels), total=len(people_uri))],
               ignore_index=True)
df

Num. results per person:


- Esa Saari                	1181
- Johan Hellman            	857
- Birgitta Norrgård        	1133
- Dani Strömbäck           	1162
- Jussi Pernaa             	1128
- Camilla Boije            	1040
- Ann-Helen Påfs-Streng    	1010
- Camilla Forsén-Ström     	917
- Christina Staffans       	838
- Antti Kuusiniemi         	735



,p,media,start,end,locator,person,person_uri
0,http://data.memad.eu/yle/stromso/00b0a0042e8b8...,http://data.memad.eu/yle/stromso/26ae6733edf02...,488.48,774.16,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari
1,http://data.memad.eu/yle/stromso/00f87ff8139cc...,http://data.memad.eu/yle/stromso/23a1d83917da5...,50.04,354.84,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari
2,http://data.memad.eu/yle/stromso/00f87ff8139cc...,http://data.memad.eu/yle/stromso/23a1d83917da5...,50.40,354.84,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari
3,http://data.memad.eu/yle/stromso/0205c04830ee2...,http://data.memad.eu/yle/stromso/8cb503afffc02...,1069.76,1176.68,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari
4,http://data.memad.eu/yle/stromso/03047c594882f...,http://data.memad.eu/yle/stromso/28d37798e22e7...,1144.08,1182.84,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari
...,...,...,...,...,...,...,...
9996,http://data.memad.eu/yle/stromso/f8f71961ab06c...,http://data.memad.eu/yle/stromso/91aabce9c2ca3...,731.16,1172.48,https://platform.limecraft.com/api/production/...,Antti Kuusiniemi,http://data.memad.eu/agent/antti-kuusiniemi
9997,http://data.memad.eu/yle/stromso/f97b9de60c3cc...,http://data.memad.eu/yle/stromso/556d1b6d332dd...,90.44,107.44,https://platform.limecraft.com/api/production/...,Antti Kuusiniemi,http://data.memad.eu/agent/antti-kuusiniemi
9998,http://data.memad.eu/yle/stromso/fd3f4b21faa27...,http://data.memad.eu/yle/stromso/8441af411cc43...,1677.36,1790.12,https://platform.limecraft.com/api/production/...,Antti Kuusiniemi,http://data.memad.eu/agent/antti-kuusiniemi
9999,http://data.memad.eu/yle/stromso/fe709b76e9dba...,http://data.memad.eu/yle/stromso/6a40aaf8b7b1e...,46.76,90.88,https://platform.limecraft.com/api/production/...,Antti Kuusiniemi,http://data.memad.eu/agent/antti-kuusiniemi


In [102]:
df['media'].value_counts()

http://data.memad.eu/yle/tahtien-ja-tahteiden-parhaat/a9450801b19248dfbc235b8f2819f908fa45cf6e     238
http://data.memad.eu/yle/tahtia-ja-tahteita-stromsossa/28652806459206a50db17babad90194c2fb4f464    238
http://data.memad.eu/yle/tahtien-ja-tahteiden-parhaat/dc5db53cf1486fead37993cb29c8e5126897b856     238
http://data.memad.eu/yle/tahtien-ja-tahteiden-parhaat/b69d5cefe688209591c0c3e8d35e6c4bfff289bd     238
http://data.memad.eu/yle/stromso/3d50256d06c83d3f3f23bbfd5d887acf8e7dd5f7                          168
                                                                                                  ... 
http://data.memad.eu/yle/obs-debatt/a8b8ca09fca8354c746e926e82287dc9dbc46278                         2
http://data.memad.eu/yle/lautasantennien-takaa/35fa8816d52d629e7a293fc1feedd5f1de701469              2
http://data.memad.eu/yle/jalkivaanto/1a242ec8d7f537af5b132acb0fe853cdb0ac964a                        2
http://data.memad.eu/yle/obs-debatt/6ef3bbbf11535f6103063a5fad064d200dac0

In [103]:
df['p'].value_counts()

http://data.memad.eu/yle/stromso/c7fd5e7cb2d525e883c3d2f262a0fd97becd40c6                       36
http://data.memad.eu/yle/stromso/46547840ad2d0f25f4a729ae6bb60694596850c6                       36
http://data.memad.eu/yle/stromso/f55aedc6c3ab1610fe9c9a2a1a74f996f591b489                       32
http://data.memad.eu/yle/stromso/2909d9c5e1a1db711d273892909f14a5f8708fa2                       32
http://data.memad.eu/yle/stromso/89070b13a07ba1fb78ea1e1bd410b0a9e6c0a01f                       32
                                                                                                ..
http://data.memad.eu/yle/euroopasta-olisi-puhuttava/15d23043b4568e2d609a724e3b6f6ea22c271ef9     1
http://data.memad.eu/yle/obs-debatt/70b79a9115fa8aded5f60c62e559a56f5a3cc39d                     1
http://data.memad.eu/yle/narbild/aeef6b9ec22f9f0d96d0be5fa3f4e12e58c188f3                        1
http://data.memad.eu/yle/narbild/7ec389de908ba08afc9d410e93cce3b4ae15edc2                        1
http://dat

In [104]:
df['duration'] = df['end'] - df['start']
df['duration'].describe()

count    10001.000000
mean       195.424436
std        179.671887
min          0.039000
25%         49.320000
50%        166.800000
75%        300.400000
max       2344.600000
Name: duration, dtype: float64

In [117]:
df_subset = df[df['duration'] < 50]
df_subset = df_subset[df_subset['duration'] > 20]
print('Subset length: %d' %len(df_subset))

df_subset['person_uri'].value_counts()

Subset length: 1294


http://data.memad.eu/agent/dani-stromback           161
http://data.memad.eu/agent/jussi-pernaa             158
http://data.memad.eu/agent/birgitta-norrgard        150
http://data.memad.eu/agent/esa-saari                141
http://data.memad.eu/agent/camilla-boije            140
http://data.memad.eu/agent/ann-helen-pafs-streng    139
http://data.memad.eu/agent/camilla-forsen-strom     110
http://data.memad.eu/agent/johan-hellman            103
http://data.memad.eu/agent/antti-kuusiniemi          99
http://data.memad.eu/agent/christina-staffans        93
Name: person_uri, dtype: int64

In [118]:
from src.connectors import memad_connector as memad
from src.connectors import limecraft_connector as limecraft

old_loc = ''
fc = None

df_subset.sort_values(by='media', ascending=False)

table = df_subset

def ex_frame(row):
    global fc
    global old_loc
    
    loc = memad.get_locator_for(row['media'])
    if not loc:
        print('No loc for: '+ row['media'])
        return 0
    
    loc = limecraft.locator2video(loc['locator']['value'])
    if loc != old_loc:
        fc = FrameCollector(loc, 'memad_gt2', id=row['media'])
        old_loc = loc
    
    start = row['start']
    end = row['end']
    duration = end - start
    quarter = int(duration / 4)
    quarter1 = start + quarter 
    center = start + quarter * 2
    quarter2 = start + quarter * 3
    
    try:
        x = fc.run(frame_no=quarter1)
        if x == 0:
            x = fc.run(frame_no=center)
        if x == 0:
            x = fc.run(frame_no=quarter2)
        return x
    except:
        print('Error: ' + loc)
        return 0

frames = [ex_frame(row) for idx, row in tqdm(table.iterrows(), total=len(table))]

W0727 17:12:18.319859 4529876416 def_function.py:598] 5 out of the last 4882 calls to <function Model.make_predict_function.<locals>.predict_function at 0x14ab2f0e0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
W0727 17:12:18.409008 4529876416 def_function.py:598] 6 out of the last 4883 calls to <function Model.make_predict_function.<locals>.predict_function at 0x14ab2f0e0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option t

In [120]:
frames = [l if l is not None else 0 for l in frames]

df_subset = df_subset.reset_index(drop=True)

df_subset['frame'] = frames
df_extracted = df_subset[df_subset['frame'] != 0]
df_extracted

,p,media,start,end,locator,person,person_uri,duration,frame
7,http://data.memad.eu/yle/stromso/16557519a0f8f...,http://data.memad.eu/yle/stromso/849bbd3b5da9c...,991.04,1038.24,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari,47.20,./frames/memad_gt2/stromso_849bbd3b5da9c01ca84...
8,http://data.memad.eu/yle/stromso/16557519a0f8f...,http://data.memad.eu/yle/stromso/849bbd3b5da9c...,991.40,1038.24,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari,46.84,./frames/memad_gt2/stromso_849bbd3b5da9c01ca84...
18,http://data.memad.eu/yle/stromso/48535f899451d...,http://data.memad.eu/yle/stromso/f0eccbac8f793...,1561.56,1589.48,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari,27.92,./frames/memad_gt2/stromso_f0eccbac8f79375f0ee...
29,http://data.memad.eu/yle/stromso/aee84fb386f92...,http://data.memad.eu/yle/stromso/bd5e495a9c696...,761.52,796.76,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari,35.24,./frames/memad_gt2/stromso_bd5e495a9c69610ae3a...
63,http://data.memad.eu/yle/stromso/0c90a724200ed...,http://data.memad.eu/yle/stromso/1dd398f54b09a...,1467.76,1498.76,https://platform.limecraft.com/api/production/...,Esa Saari,http://data.memad.eu/agent/esa-saari,31.00,./frames/memad_gt2/stromso_1dd398f54b09a4cdabf...
...,...,...,...,...,...,...,...,...,...
1236,http://data.memad.eu/yle/narbild/02c56f2c358c5...,http://data.memad.eu/yle/narbild/4b6fb8183414b...,1673.32,1706.00,https://platform.limecraft.com/api/production/...,Antti Kuusiniemi,http://data.memad.eu/agent/antti-kuusiniemi,32.68,./frames/memad_gt2/narbild_4b6fb8183414baa9d62...
1237,http://data.memad.eu/yle/narbild/11dfc167298bf...,http://data.memad.eu/yle/narbild/334eebdbd1aaf...,1517.04,1540.96,https://platform.limecraft.com/api/production/...,Antti Kuusiniemi,http://data.memad.eu/agent/antti-kuusiniemi,23.92,./frames/memad_gt2/narbild_334eebdbd1aaf4b562d...
1238,http://data.memad.eu/yle/narbild/11dfc167298bf...,http://data.memad.eu/yle/narbild/334eebdbd1aaf...,1517.40,1540.96,https://platform.limecraft.com/api/production/...,Antti Kuusiniemi,http://data.memad.eu/agent/antti-kuusiniemi,23.56,./frames/memad_gt2/narbild_334eebdbd1aaf4b562d...
1239,http://data.memad.eu/yle/narbild/15f010b5662d4...,http://data.memad.eu/yle/narbild/334eebdbd1aaf...,1692.16,1721.56,https://platform.limecraft.com/api/production/...,Antti Kuusiniemi,http://data.memad.eu/agent/antti-kuusiniemi,29.40,./frames/memad_gt2/narbild_334eebdbd1aaf4b562d...


In [121]:
df_extracted['person'].value_counts()

Johan Hellman            37
Dani Strömbäck           26
Jussi Pernaa             26
Birgitta Norrgård        20
Camilla Boije            20
Ann-Helen Påfs-Streng    18
Esa Saari                16
Camilla Forsén-Ström     15
Christina Staffans       12
Antti Kuusiniemi         12
Name: person, dtype: int64

In [122]:
df_extracted.to_csv(r'memad_parts2.csv', index = False)

### Plan

1. Extract 1 folders for each person
2. Manually remove the ones not containing the person
3. Re-import in this notebook and fix the table

##### 1. Extract 3 folders for each person

In [123]:
from shutil import copyfile

for p in df_extracted['person'].unique():
    out_path = './dataset_memad2/%s' % p.replace(' ', '_')
    os.makedirs(out_path, exist_ok=True)

    for i, row in df_extracted[df_extracted['person'] == p].iterrows():
        x = row['frame']
        fname = x.split('/')[-1]
        copyfile(x, os.path.join(out_path, fname))

##### 2. Manually move those to 2 groups: containing and not containing the person


_(This action is made offline)_

##### 3. Re-import in this notebook and fix the table

In [68]:
manual_good = []
manual_bad = []
for p in os.listdir('./dataset_memad'):
    if p == '.DS_Store':
        continue
        

    for f in os.listdir(os.path.join('./dataset_memad', p)):
        if f == '.DS_Store':
            continue
        if p == 'unknown':
            manual_bad.append('./frames/memad_gt/'+f)
        else:
            manual_good.append((p,'./frames/memad_gt/'+f))

In [69]:
unique, counts = np.unique(np.array(manual_good)[:,0], return_counts=True)
for a,b in zip(unique,counts):
    print(a, b)

Delahousse,_Laurent 10
Drucker,_Marie 18
Gastrin,_Sophie 25
Le_Saint,_Sophie 26
Lucet,_Elise 28
Rincquesen,_Nathanaël_de 65
